In [1]:
import pandas as pd
import numpy as np
from bokeh.io import curdoc, output_file, show
from bokeh.models import ColumnDataSource, HoverTool, Select
from bokeh.plotting import figure
from bokeh.layouts import row, widgetbox

In [2]:
deaths_url = "https://raw.githubusercontent.com/datasets/covid-19/master/data/countries-aggregated.csv"
ds = pd.read_csv(deaths_url)

ds["Date"] = pd.to_datetime(ds["Date"])

countries = ds.Country.values
countries = np.unique(countries).tolist()
ds.tail()

,Date,Country,Confirmed,Recovered,Deaths
35151,2020-07-26,West Bank and Gaza,10469,3752,76
35152,2020-07-26,Western Sahara,10,8,1
35153,2020-07-26,Yemen,1681,797,479
35154,2020-07-26,Zambia,4481,2350,139
35155,2020-07-26,Zimbabwe,2512,518,34


In [3]:
import geopandas as gpd

shapefile = 'shapefiles/ne_110m_admin_0_countries.shp'
gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]

gdf.columns = ['country', 'country_code', 'geometry']
gdf = gdf.drop(gdf.index[159]) # Drop Antarctica

gdfCountries = gdf.country.values

gdf.head(2)

,country,country_code,geometry
0,Fiji,FJI,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,United Republic of Tanzania,TZA,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."


In [4]:
# prints the missing and additional elements in list2  
print("Missing values in gdfCountries:", (set(countries).difference(gdfCountries))) 
  
# prints the missing and additional elements in list1 
print("Missing values in countries:", (set(gdfCountries).difference(countries))) 

Missing values in gdfCountries: {'Diamond Princess', 'Congo (Kinshasa)', 'Timor-Leste', 'Holy See', 'Liechtenstein', 'Bahrain', "Cote d'Ivoire", 'Malta', 'Saint Kitts and Nevis', 'North Macedonia', 'Grenada', 'Korea, South', 'Serbia', 'Sao Tome and Principe', 'Eswatini', 'Monaco', 'Antigua and Barbuda', 'Mauritius', 'Bahamas', 'Comoros', 'Singapore', 'Taiwan*', 'Saint Vincent and the Grenadines', 'West Bank and Gaza', 'Maldives', 'Barbados', 'Burma', 'MS Zaandam', 'Andorra', 'Seychelles', 'Dominica', 'Tanzania', 'Cabo Verde', 'San Marino', 'Congo (Brazzaville)', 'US', 'Saint Lucia'}
Missing values in countries: {'Northern Cyprus', 'Somaliland', 'Turkmenistan', 'East Timor', 'Vanuatu', 'United States of America', 'Ivory Coast', 'Palestine', 'Falkland Islands', 'United Republic of Tanzania', 'North Korea', 'Puerto Rico', 'Macedonia', 'Republic of the Congo', 'Taiwan', 'Republic of Serbia', 'New Caledonia', 'South Korea', 'Democratic Republic of the Congo', 'The Bahamas', 'eSwatini', 'Fre

In [5]:
ds.replace(to_replace ="US", value ="United States of America", inplace = True) 
ds.replace(to_replace ="Congo (Kinshasa)", value ="Democratic Republic of the Congo", inplace = True) 
ds.replace(to_replace ="Congo (Brazzaville)", value ="Republic of the Congo", inplace = True) 
ds.replace(to_replace ="Cote d'Ivoire", value ="Ivory Coast", inplace = True) 
ds.replace(to_replace ="Taiwan*", value ="Taiwan", inplace = True) 
ds.replace(to_replace ="Tanzania", value ="United Republic of Tanzania", inplace = True) 
ds.replace(to_replace ="Burma", value ="Myanmar", inplace = True) 
ds.replace(to_replace ="Bahamas", value ="The Bahamas", inplace = True) 
ds.replace(to_replace ="Serbia", value ="Republic of Serbia", inplace = True) 
ds.replace(to_replace ="Timor-Leste", value ="East Timor", inplace = True) 
ds.replace(to_replace ="Korea, South", value ="South Korea", inplace = True) 
ds.replace(to_replace ="North Macedonia", value ="Macedonia", inplace = True) 

countries = ds.Country.values
countries = np.unique(countries).tolist()

In [6]:
def subset_source(country):
    mask = ds['Country'] == country
    ds_sub = ds[mask].reset_index()
    
    source = ColumnDataSource(data={
        'date' : ds_sub.Date,
        'country' : ds_sub.Country,
        'confirmed' : ds_sub.Confirmed,
        'recovered' : ds_sub.Recovered,
        'dead' : ds_sub.Deaths
    })
    
    return source, ds_sub

In [7]:
selectedCountry = "India"
source, ds_sub = subset_source(selectedCountry)

plot = figure(title = selectedCountry, x_axis_label = 'Dates', y_axis_label = 'Cases Count', x_axis_type = 'datetime', plot_width=700, plot_height=700)

output_file('graph.html')

select = Select(title="Country : ", value="India", options=countries)

plot.line(x = 'date', y = 'confirmed', source = source, color = 'blue')
plot.line(x = 'date', y = 'recovered', source = source, color = 'green')
plot.line(x = 'date', y = 'dead', source = source, color = 'red')

hover = HoverTool(tooltips = [('Confirmed', '@confirmed'), ('Recovered', '@recovered'), ('Deceased', '@dead')])
plot.add_tools(hover)

layout = row(widgetbox(select), plot)
show(layout)

In [8]:
def subset_source_map(date):
    dateSub = ds[ds['Date'] == date]
 
    source = ColumnDataSource(data={
        'date' : dateSub.Date,
        'country' : dateSub.Country,
        'confirmed' : dateSub.Confirmed,
        'recovered' : dateSub.Recovered,
        'dead' : dateSub.Deaths
    })
    
    return source, dateSub

In [22]:
import json
from datetime import datetime

selectedDate = '2020-06-22'
source, dateSub = subset_source_map(selectedDate)

merged = gdf.merge(dateSub, left_on = 'country', right_on = 'Country', how = 'left')
merged['Date'] = merged['Date'].dt.strftime('%Y-%m-%d')

merged_json = json.loads(merged.to_json()) # Read data to JSON
json_data = json.dumps(merged_json)

In [28]:
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer

geosource = GeoJSONDataSource(geojson = json_data)

palette = brewer["YlGnBu"][6]
palette = palette[::-1]

color_mapper = LinearColorMapper(palette = palette, low = dateSub['Confirmed'].min(), 
                                 high = (dateSub['Confirmed'].max()/3.5), nan_color = '#d9d9d9')

hover = HoverTool(tooltips = [ ('Country','@Country'),('Confirmed Cases','@Confirmed')])

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
border_line_color= None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)

p = figure(title = 'Number of COVID-19 cases', plot_height = 600 , plot_width = 1200, toolbar_location = None, tools = [hover])
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

#Add patch renderer to figure.
p.patches('xs', 'ys', source = geosource, fill_color = {'field' :'Confirmed', 'transform' : color_mapper}, 
          line_color = 'black', line_width = 0.25, fill_alpha = 1)

#Specify layout
p.add_layout(color_bar, 'below')

show(p)